# Instrumental Variables
Basic exploration of IVs

Say we have some underlying population trends which we cannot observe. These trends relate to how people after domestic diputes respond to being 'arrested' or being 'coddled'. The numbers in this table are the liklihood that a domestic violence abuser will abuse again (recidivism) given some tretment and given whether they are violent or not.

Treatment | Normal Person | Violent Person
-|-|-
C | 0.1 | 0.15
A | 0.02 | 0.05

We have some data which we **do** know, but which is tainted by the fact that in every case a violent person in the 'coddled' group ('C') will assault the police officer and thus need to be arrested, and that this happens 10% of the time. However we still want to know the underlying treatment effect on the non-violent;

Effect of treatment (C) on outcome (Y) = Effect of IV (Z) on outcome (Y) / Effect of treatment (C) on IV (Z)

Now say we observe the experiment which the above (unknown) table implies, with 100 people in each group:

In [1]:
rate_coddled_group = 0.1 * 0.9 + 0.1 * 0.05
rate_arrested_group = 0.02 * 0.9 + 0.1 * 0.05
print("C:", round(rate_coddled_group, 4), "\nA:" , round(rate_arrested_group, 4))

C: 0.095 
A: 0.023


This is the effect of our intrumental variable (i.e. the dummy for whether in the coddled (C) or arrested (A) group.

'Effect of IV on outcome' is the difference between the outcome (recidivism rates) in the C and A groups.
'Effect of treatment on IV' is the liklihood that a treated case (coddled) will actually be treated.  
Using our IV formula:

In [2]:
treatment_effect = (rate_coddled_group - rate_arrested_group) / (0.9)
print(round(treatment_effect, 4))

0.08


This should match the table at the top (i.e. [Normal Person : C] - [Normal Person : A])

# IV on actual data:

### The equations for the 2SLS with controls:
Reduced Form (effect of the insrument on the outcome):
$$Y_i = \alpha_0 + \rho Z_i + \gamma_0 A_i + e_0i$$
First Stage (effect of the instrument on the treatment):
$$D_i = \alpha_1 + \phi Z_i + \gamma_1 A_i + e_1i$$

2SLS ():
$$\hat{D}_i = \alpha_1 + \phi Z_i + \gamma_1 A_i$$
Finally (effect of treatment on the outcome):
$$Y_i = \alpha_2 + \lambda_{2SLS} \hat{D}_i + \gamma_2 A_i + e_2i$$

**What does this mean?**  
We know the effect of $Z_i$ on $Y_i$ but we want to know the effect of $D_i$ on $Y_i$ (controlling for $A_i$). Thus we get the 

This final regression gives us the treatment effect ($\lambda_{2SLS}$)

Variable | Meaning
- | -
$Y_i$ | Outcome of interest
$Z_i$ | Instrument
$A_i$ | Thing(s) we are controlling for
$D_i$ | Treatment variable


In [3]:
import pandas as pd

lss = pd.read_csv('LifeCycleSavings.csv')

lss.sample(5)

,Unnamed: 0,sr,pop15,pop75,dpi,ddpi
17,Honduras,7.70,47.32,0.58,232.44,3.19
47,Uruguay,9.24,28.13,2.72,766.54,1.88
27,Netherlands,14.65,24.71,3.25,1740.70,7.66
36,South Rhodesia,13.30,31.92,1.52,250.96,2.00
33,Philippines,12.78,46.26,1.12,152.01,2.00


In [4]:
#Add a > 100 dpi dummy
lss['dpi_100_plus'] = lss['dpi'] > 1000

#Add a > 8 sr dummy
lss['sr_8_plus'] = lss['sr'] > 8

#Add a pop75 > 4 dummy
lss['pop75_above_mean'] = lss['pop75'] > lss['pop75'].mean()

#Add a pop75 > 4 dummy
lss['pop15_above_mean'] = lss['pop15'] > lss['pop15'].mean()

lss.sample(5)

,Unnamed: 0,sr,pop15,pop75,dpi,ddpi,dpi_100_plus,sr_8_plus,pop75_above_mean,pop15_above_mean
33,Philippines,12.78,46.26,1.12,152.01,2.00,False,True,False,True
34,Portugal,12.49,28.96,2.85,579.51,7.48,False,True,True,False
35,South Africa,11.14,31.94,2.28,651.11,2.19,False,True,False,False
46,Jamaica,7.72,41.12,1.73,380.47,10.23,False,False,False,True
27,Netherlands,14.65,24.71,3.25,1740.70,7.66,True,True,True,False


In [5]:
#Let's run a regression of sr on pop_75_plus

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(lss.ix[:, ['dpi_100_plus']], lss['sr'])
print('beta:', lr.coef_[0], 'alpha:', lr.intercept_)

beta: 2.96583333333 alpha: 8.48466666667


In [6]:
#Not sure if this even makes sence but let's use two correlated columns as a treatment and IV respectively
lss.corr()

,sr,pop15,pop75,dpi,ddpi,dpi_100_plus,sr_8_plus,pop75_above_mean,pop15_above_mean
sr,1.000000,-0.455538,0.316521,0.220359,0.304787,0.327583,0.815771,0.320013,-0.470311
pop15,-0.455538,1.000000,-0.908479,-0.756188,-0.047826,-0.780344,-0.433544,-0.872910,0.938793
pop75,0.316521,-0.908479,1.000000,0.787000,0.025321,0.820779,0.326212,0.896571,-0.862022
dpi,0.220359,-0.756188,0.787000,1.000000,-0.129486,0.873374,0.186164,0.770809,-0.725176
ddpi,0.304787,-0.047826,0.025321,-0.129486,1.000000,-0.074608,0.203665,-0.004543,-0.011368
dpi_100_plus,0.327583,-0.780344,0.820779,0.873374,-0.074608,1.000000,0.272166,0.768122,-0.753592
sr_8_plus,0.815771,-0.433544,0.326212,0.186164,0.203665,0.272166,1.000000,0.386976,-0.478199
pop75_above_mean,0.320013,-0.872910,0.896571,0.770809,-0.004543,0.768122,0.386976,1.000000,-0.886750
pop15_above_mean,-0.470311,0.938793,-0.862022,-0.725176,-0.011368,-0.753592,-0.478199,-0.886750,1.000000


The pop_75 dummy and the dpi dummy are highly correlated. Let's use these:

Step 1) Get the effect of the population > 75 on the outcome (sr):

Step 2) Get the effect of the population > 75 on the disposable income dummy (dpi_100_plus)

**NOTE**  
Y = 'sr'  
Z = 'pop_75_above_mean'  
D = 'dpi_100_plus'  
A = 'sr_8_plus'

In [7]:
#2SLS:
lr_iv_y = LinearRegression()    
lr_iv_d = LinearRegression()

lr_iv_y.fit(lss.ix[:, ['pop75_above_mean']], lss['sr'])
lr_iv_d.fit(lss.ix[:, ['pop75_above_mean']], lss['dpi_100_plus'])

print('LATE:', lr_iv_y.coef_[0] / lr_iv_d.coef_[0])
print(lr_iv_y.coef_[0], lr_iv_d.coef_[0])

LATE: 3.77191489362
2.84102564103 0.753205128205


In [8]:
#2SLS METHOD 2
lr_iv_y = LinearRegression()    
lr_iv_d = LinearRegression()

#Run first regression
lr_iv_d.fit(lss.ix[:, ['pop75_above_mean']], lss['dpi_100_plus'])

#Save fitted values
lss['D_fitted'] = lr_iv_d.predict(lss.ix[:, ['pop75_above_mean']])

#run on fitted values
lr_iv_y.fit(lss.ix[:, ['D_fitted']], lss['sr'])

print('Lambda:', lr_iv_y.coef_[0])

Lambda: 3.77191489362


This lambda matches the LATE calculated earlier!!!

Now let's add some controls:

In [9]:
#2SLS WITH CONTROLS STEP BY STEP REGRESSION

lr_iv_b = LinearRegression()
lr = LinearRegression()

#Run first regression
lr_iv_b.fit(X = lss.ix[:, ['sr_8_plus']], y = lss['pop75_above_mean'])

predictions = lr_iv_b.predict(lss.ix[:, ['sr_8_plus']])
lss['residuals_Z'] = lss['pop75_above_mean'] - predictions

rho = lr.fit(lss.ix[:, ['residuals_Z']], lss['sr']).coef_[0]
phi = lr.fit(lss.ix[:, ['residuals_Z']], lss['dpi_100_plus']).coef_[0]

print("rho:", rho, "phi:", phi)
print("LATE:", rho / phi)

rho: 0.0451963350785 phi: 0.764397905759
LATE: 0.0591267123288


In [10]:
#2SLS WITH CONTROLS STEP BY COVARIANCE ONLY

import numpy as np

num = np.cov(lss['sr'], lss['residuals_Z'])[0][1]
denom = np.cov(lss['dpi_100_plus'], lss['residuals_Z'])[0][1]

print("numerator:", num, "denominator:", denom)
print("LATE:", num / denom)

numerator: 0.00978741496599 denominator: 0.165532879819
LATE: 0.0591267123288


In [11]:
#2SLS WITH CONTROLS ALL REGRESSION

lr_iv_y = LinearRegression()    
lr_iv_d = LinearRegression()

#Run first regression
lr_iv_d.fit(lss.ix[:, ['pop75_above_mean', 'sr_8_plus']], lss['dpi_100_plus'])

#Save fitted values
lss['D_fitted'] = lr_iv_d.predict(lss.ix[:, ['pop75_above_mean', 'sr_8_plus']])

#run on fitted values
lr_iv_y.fit(lss.ix[:, ['D_fitted', 'sr_8_plus']], lss['sr'])

print('lambda_2SLS:', lr_iv_y.coef_[0])

lambda_2SLS: 0.0591267123288


These three methods are identical too :)

## IV standard Errors

### Restating the equations for the 2SLS with controls:
Reduced Form:
$$Y_i = \alpha_0 + \rho Z_i + \gamma_0 A_i + e_0i$$
First Stage:
$$D_i = \alpha_1 + \phi Z_i + \gamma_1 A_i + e_1i$$

2SLS:
$$\hat{D}_i = \alpha_1 + \phi Z_i + \gamma_1 A_i$$
Finally:
$$Y_i = \alpha_2 + \lambda_{2SLS} \hat{D}_i + \gamma_2 A_i + e_2i$$

This final regression gives us the treatment effect ($\lambda_{2SLS}$)

### Standard Errors
First, we need to find the residuals:
$$\eta_i = Y_i - \alpha_2 - \lambda_{2SLS} D_i - \gamma_2 A_i$$
Note that we use $D_i$ and not $\hat{D}_i$. 

Then, the standard errors are:
$$SE(\hat{\lambda}_{2SLS}) = \frac{\sigma_{\eta}}{\sqrt{n}} * \frac{1}{\sigma_\hat{D}}$$

Note here that $\sigma_{\hat(D)_i}$ is the Standard Deviation of the $\hat{D}_i$ regression above and not the Standard Deviation of the first stage. This is part of why it's usually best to calculate these kinds of Standard Errors with (in Joshua Angrist's words) *"professional software"*.

**NOTE**  
Y = 'sr'  
Z = 'pop_75_above_mean'  
D = 'dpi_100_plus'  
A = 'sr_8_plus'

In [12]:
#Code to calculate Standard Error of residuals:
eta_i = lss['sr'] - lr_iv_y.intercept_ - lr_iv_y.coef_[0] * lss['dpi_100_plus'] - lr_iv_y.coef_[1] * lss['sr_8_plus']

In [17]:
import scipy as sp
SE_lamda_2SLS = sp.std(eta_i) / sp.sqrt(len(eta_i)) / sp.std(lss['dpi_100_plus'])

In [18]:
SE_lamda_2SLS

0.73905826481333581